In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%matplotlib inline

from matplotlib import pyplot as plt
import psycopg2 as pg
import pandas.io.sql as psql
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
import folium
import numpy as np
import openpyxl
from IPython.display import display
from IPython.display import HTML

# get connected to the database
conn = pg.connect("dbname=uwwtd_eu port=5432 user=postgres password=postgres host=192.168.1.60")
dataframe = psql.read_sql("SELECT * FROM calcul.prio_agg_matrix_new", conn)
dataframe.to_csv(r'data/calcul_prio_agg_matrix_v4.csv', index = False)
dataframe.to_excel(r'data/calcul_prio_agg_matrix_v4.xlsx', index = False)

#==================================== Custom functions
def get_dtt_score(val, load, ref_rate=1):
    if val>0:
        rate = 100 * val/load
        if val >= 10000 :
            return 4
        elif val > 2000 and val < 10000 and rate > ref_rate :
            return 3
        elif val > 2000 and rate <= ref_rate :
            return 2
        else:
            return 1 
    else:
        return 0

def get_dtt_aggravation(dtt, dtt_n1, load, ref_rate=1) :
    score = get_dtt_score(dtt, load, ref_rate)
    if dtt_n1 < dtt:
        return score
    else:
        return score + 1
    
def get_ponderate_score(s_art3, s_art4, s_art5, w_art3=5, w_art4=3, w_art5=2):
    w_sum = w_art3 + w_art4 + w_art5
    score = (s_art3*w_art3 + s_art4*w_art4 + s_art5*w_art5)/w_sum
    return score

def filter_wfd_chemicalstatus(x):
    if x > 2:
        return x
    else:
        return 0

def filter_wfd_ecologicalstatus(x):
    if x > 3:
        return x
    else:
        return 0

def get_sensitive_cat(row):
    sa_type = row['typeofsensitivearea']
    cat = 0
    if sa_type == 'LSA':
        cat = 1
    elif sa_type == 'NA':
        cat = 2
    else:
        cat = 3
    cat = cat + row['spztype_freshwaterFishDesignatedWater']+row['spztype_shellfishDesignatedWater']+row['spztype_drinkingWaterProtectionArea']+row['spztype_nitrateVulnerableZone']
    return cat

#==================================== Main function
#Load data from SQL DB
def prepare_report(country = "all", map_display=False,
                   crit1_w = 1, crit2_w = 1, crit3_w = 1, crit4_w = 1,
                   dtt_art3_w = 5, dtt_art4_w = 3, dtt_art5_w = 2,
                   wfd_p11_w = 5, wfd_p12_w = 5, wfd_p26_w = 5,
                   
                  ):
    #=== default settings ==========================================================================================
    year = 2016
    countries = {'bg':'Bulgaria', 'cy':'Cyprus','cz':'Czech-Republic', 'el':'Greece','es':'Spain','fr':'France','hr':'Croatia', 'hu':'Hungary', 'it':'Italia','lv':'Latvia','mt':'Malta','pl':'Poland', 'pt':'Portugal', 'ro':'Romania', 'si':'Slovenia','sk':'Slovakia', 'ie':'Ireland' }

    #=== Load data =================================================================================================
    dataframe = pd.read_csv("data/calcul_prio_agg_matrix_v4.csv")
    #Filter by country code
    if country != 'all':
        dataframe = dataframe[dataframe.cc == country]
        country_name = countries[country]
        default_map_zoom = 7
    else:
        country_name = 'all countries'
        default_map_zoom = 2
    
    #=== Prepare data ==============================================================================================
    #======= CRITERION 1
    #============== Distance to target from 10th reporting    
    dataframe['dtt_art3_score'] = dataframe.apply(lambda row : get_dtt_score(row['dtt3_10th'], row['generatedload_10th'], 2), axis = 1)     
    dataframe['dtt_art4_score'] = dataframe.apply(lambda row : get_dtt_score(row['dtt4p_10th'], row['connectedload_10th'], 1), axis = 1)     
    dataframe['dtt_art5_score'] = dataframe.apply(lambda row : get_dtt_score(row['dtt5p_10th'], row['connectedload_10th'], 1), axis = 1)     

    #============== Distance to target from 10th reporting with “aggravation factor” from 9th report
    dataframe['dtt_art3_aggravation_score'] = dataframe.apply(lambda row : get_dtt_aggravation(row['dtt3_10th'], row['dtt3_9th'], row['generatedload_10th'], 2), axis = 1) 
    dataframe['dtt_art4_aggravation_score'] = dataframe.apply(lambda row : get_dtt_aggravation(row['dtt4p_10th'], row['dtt4p_9th'], row['connectedload_10th'], 1), axis = 1)
    dataframe['dtt_art5_aggravation_score'] = dataframe.apply(lambda row : get_dtt_aggravation(row['dtt5p_10th'], row['dtt5p_9th'], row['connectedload_10th'], 1), axis = 1)

    #criterion 1 score
    dataframe['criterion_1_dtt_score'] = dataframe.apply(lambda row : get_ponderate_score(
            row['dtt_art3_aggravation_score'],
            row['dtt_art4_aggravation_score'],
            row['dtt_art5_aggravation_score'],
            dtt_art3_w,
            dtt_art4_w,
            dtt_art5_w
    ), axis = 1) 
    
    #======= CRITERION 2
    #============== wfd_chemicalstatus (disabled)
    wfd_chemicalstatus = pd.to_numeric(dataframe['wfd_chemicalstatus'], errors='coerce')
    np.nan_to_num(wfd_chemicalstatus, False)
    wfd_chemicalstatus = wfd_chemicalstatus.apply(lambda row : filter_wfd_chemicalstatus(row))

    #============== wfd_ecologicalstatus
    wfd_ecologicalstatus = pd.to_numeric(dataframe['wfd_ecologicalstatus'], errors='coerce')
    np.nan_to_num(wfd_ecologicalstatus, False)
    wfd_ecologicalstatus = wfd_ecologicalstatus.apply(lambda row : filter_wfd_ecologicalstatus(row))
    
    #============== criterion_2_collect
    criterion_2_collect = dataframe['wfd_uwwtd_status_diffuse_p26'] * wfd_p26_w
    np.nan_to_num(criterion_2_collect, False)
    
    #============== wfd_uwwtd_status_point_p11
    wfd_uwwtd_status_point_p11 = dataframe['wfd_uwwtd_status_point_p11']*wfd_p11_w
    np.nan_to_num(wfd_uwwtd_status_point_p11, False)
    
    #============== wfd_uwwtd_status_point_p12
    wfd_uwwtd_status_point_p12 = dataframe['wfd_uwwtd_status_point_p12']*wfd_p12_w
    np.nan_to_num(wfd_uwwtd_status_point_p12, False)
    
    #============== nat_2000
    nat_2000 = dataframe['near_natura2000_uww_impacted'] * dataframe['near_natura2000_uww_impacted_intensity']
    np.nan_to_num(nat_2000, False)

    dataframe['criterion_2_collect'] = criterion_2_collect
    #TODO split wfd & natura 2000
    dataframe['criterion_2_nat2000'] = nat_2000
    dataframe['criterion_2_wfd'] = np.nansum([
            wfd_uwwtd_status_point_p11, 
            wfd_uwwtd_status_point_p12, 
            #wfd_chemicalstatus, #disabled in this version
            wfd_ecologicalstatus
        ], axis = 0)

    dataframe['criterion_2'] = dataframe['criterion_2_collect'] + dataframe['criterion_2_wfd'] + dataframe['criterion_2_nat2000']    
    
    #======= CRITERION 3 - Sensitive rank
    dataframe['sensitive_cat'] = dataframe.apply(lambda row : get_sensitive_cat(row), axis = 1)
    
    #======= Final rank
    #TODO :
    # - use spzone type in scoring (designated water & vulnerable area)
    dataframe['final_rank'] = ((dataframe['criterion_1_dtt_score']*crit1_w) + \
        (dataframe['criterion_2'] * crit2_w) + \
        (dataframe['sizecat'] * crit3_w) + \
        (dataframe['sensitive_cat'] * crit4_w)/(crit1_w+crit2_w+crit3_w+crit4_w))
    
    
    #=== Display the report ========================================================================================
    display(HTML("<h1>Dataset</h1>"))
    #======= Dataset preview table
    display(HTML("<h2>data preview</h2>"))
    print("10 first lines of the dataset")
    display(dataframe.head(10))
    #dataframe.columns
    
    #======= Dataset distribution
    display(HTML("<h2>dataset distribution</h2>"))
    plt.figure()
    fig, ax = plt.subplots()
    ax.hist(dataframe.sizecat, color='orange', align='mid', rwidth=1)
    plt.xlabel('category')
    plt.xticks([1, 2, 3, 4])
    plt.ylabel('Number of agglomeration')
    plt.title(f'Non compliant Agglomeration by category size  in {country_name}')
    plt.show()
    
    #======= Map
    if map_display:
        display(HTML("<h2>Map of selected agglomeration</h2>"))
        lat_mean = np.mean(dataframe.y)
        lon_mean = np.mean(dataframe.x)
        print("Map center (lat, lon)", lat_mean, lon_mean)

        map = folium.Map(
            tiles='OpenStreetMap',
            location=(lat_mean, lon_mean),
            zoom_start=default_map_zoom
        )
        folium.LayerControl().add_to(map)
        map_tooltip = 'Click me!'
        #Add markers on map
        for lat, lon, aggcode, aggname, generatedload in zip(dataframe.y, dataframe.x, dataframe.aggcode, dataframe.aggname, dataframe.generatedload_10th):
            folium.Marker(
                [lat, lon], 
                popup=f'<b><a target="_blank" href="https://uwwtd.eu/{country_name}/agglomeration/{aggcode}/{year}">{aggname} [{aggcode}]</a></b><br/><b>Generated load : </b>{generatedload} p.e.<br/>',
                tooltip=map_tooltip
            ).add_to(map)

        display(map)
        
    display(HTML("<h1>Prioritization</h1>"))
    #======= Dataset preview table
    display(HTML("<h2>Criterion 1</h2><p>You can change weight of each parameter of this criterion by using Art3, Art4 & Art5 DTT weight slider</p>"))
    display(
        dataframe[['aggcode','aggname','sizecat','generatedload_10th', 'dtt_art3_aggravation_score', 'dtt_art4_aggravation_score', 'dtt_art5_aggravation_score','criterion_1_dtt_score']]
            .sort_values(by=['criterion_1_dtt_score', 'generatedload_10th'], ascending=False)
            .head(10)
    )
    plt.figure()
    fig, ax = plt.subplots()
    ax.hist(dataframe.criterion_1_dtt_score, color='blue', align='mid', rwidth=1)
    plt.xlabel('score')
    #plt.xticks([1, 2, 3, 4])
    plt.ylabel('Number of agglomeration')
    plt.title(f'Number of agglomeration by Criterion 1 score in {country_name}')
    plt.show()
    
    plt.figure()
    fig, ax = plt.subplots()
    ax.scatter(y=dataframe.generatedload_10th , x=dataframe.criterion_1_dtt_score, c='red')
    plt.xlabel('score')
    #plt.xticks([1, 2, 3, 4])
    plt.ylabel('Size of agglomeration')
    plt.title(f'Scatter of agglomeration by size and criterion 1 score in {country_name}')
    plt.show()
    
    display(HTML("<h1>Criterion 2 – Environmental impact</h1>"))
    display(
        dataframe[['aggcode','aggname','sizecat','generatedload_10th', 'criterion_2_collect', 'criterion_2_wfd', 'criterion_2_nat2000','criterion_2']]
            .sort_values(by=['criterion_2', 'generatedload_10th'], ascending=False)
            .head(10)
    )
    
    plt.figure()
    fig, ax = plt.subplots()
    ax.scatter(y=dataframe.generatedload_10th , x=dataframe.criterion_2, c='olive')
    plt.xlabel('score')
    #plt.xticks([1, 2, 3, 4])
    plt.ylabel('Size of agglomeration')
    plt.title(f'Scatter of agglomeration by size and criterion 2 score in {country_name}')
    plt.show()
    
    display(HTML("<h1>Criterion 3 - Sensitive rank</h1>"))
    display(
        dataframe[['aggcode','aggname','sizecat', 'generatedload_10th','typeofsensitivearea','spztype_freshwaterFishDesignatedWater','spztype_shellfishDesignatedWater','spztype_drinkingWaterProtectionArea','spztype_nitrateVulnerableZone','sensitive_cat']]
        .sort_values(by=['sensitive_cat', 'generatedload_10th'], ascending=False)
        .head(10)
    )
    plt.figure()
    fig, ax = plt.subplots()
    ax.scatter(y=dataframe.generatedload_10th , x=dataframe.sensitive_cat, c='purple')
    plt.xlabel('score')
    #plt.xticks([1, 2, 3, 4])
    plt.ylabel('Size of agglomeration')
    plt.title(f'Scatter of agglomeration by size and criterion 3 score in {country_name}')
    plt.show()
    
    display(HTML("<h1>Final rank</h1>"))
    final = dataframe[[
        'aggcode','aggname','generatedload_10th','sizecat',
        'criterion_1_dtt_score',
        'criterion_2',
        'sensitive_cat',
        'final_rank'
    ]].sort_values(by=['final_rank', 'generatedload_10th'], ascending=False)

    display(final.head(50))
    
    plt.figure()
    fig, ax = plt.subplots()
    ax.scatter(y=dataframe.generatedload_10th , x=dataframe.final_rank, c='green')
    plt.xlabel('score')
    #plt.xticks([1, 2, 3, 4])
    plt.ylabel('Size of agglomeration')
    plt.title(f'Scatter of agglomeration by size and final score score in {country_name}')
    plt.show()
    
    final.to_csv(r'output/ranking_'+ country +'_v4.csv', sep="\t", index = False)
    final.to_excel(r'output/ranking_'+ country +'_v4.xlsx', index = False)
    return 'end'
 

    
    
#========== Starter
dataframe = interact(prepare_report,
        country =widgets.Dropdown(options=['all', 'bg', 'cy', 'cz', 'el', 'es', 'fr', 'hr', 'hu', 'it', 'lv', 'mt', 'pl', 'pt', 'ro', 'si', 'sk', 'ie'],value='cz',description='Country:'),
        map_display = widgets.Checkbox(value=False,description='Display the map'),
        crit1_w = widgets.IntSlider(value=1,min=0,max=10,step=1,description='Criterion 1 weight:', style={'description_width': 'initial'}),
        crit2_w = widgets.IntSlider(value=1,min=0,max=10,step=1,description='Criterion 2 weight:', style={'description_width': 'initial'}),
        crit3_w = widgets.IntSlider(value=1,min=0,max=10,step=1,description='Size category weight:', style={'description_width': 'initial'}),
        crit4_w = widgets.IntSlider(value=1,min=0,max=10,step=1,description='Criterion 1 weight:', style={'description_width': 'initial'}),             
        dtt_art3_w = widgets.IntSlider(value=5,min=1,max=10,step=1,description='Art3 DTT weight:', style={'description_width': 'initial'}),
        dtt_art4_w = widgets.IntSlider(value=3,min=1,max=10,step=1,description='Art4 DTT weight:', style={'description_width': 'initial'}),
        dtt_art5_w = widgets.IntSlider(value=2,min=1,max=10,step=1,description='Art5 DTT weight:', style={'description_width': 'initial'}),
        wfd_p11_w = widgets.IntSlider(value=5,min=1,max=10,step=1,description='WFD pressure 1.1:', style={'description_width': 'initial'}),
        wfd_p12_w = widgets.IntSlider(value=5,min=1,max=10,step=1,description='WFD pressure 1.2:', style={'description_width': 'initial'}),
        wfd_p26_w = widgets.IntSlider(value=5,min=1,max=10,step=1,description='WFD pressure 2.6:', style={'description_width': 'initial'})
    )


interactive(children=(Dropdown(description='Country:', index=3, options=('all', 'bg', 'cy', 'cz', 'el', 'es', …